In [ ]:
import numpy as np
import hashlib
import math
import pandas as pd 
import matplotlib.pyplot as plt
import random
import re
import string
from nltk.tokenize import WordPunctTokenizer

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.decomposition import TruncatedSVD, PCA
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.svm import SVC
import spacy

## Q1.

It's 2020 and every second you spend in quarantine is driving you crazy. The pandemic would already be enough, but it seems that day in, day out, people are commenting on another disaster or weird event happening around the globe. Well, at least according to twitter. 

<img src="media/biblical.png" width="50%" />


_"Maybe I shouldn't get my news from twitter..."_, you wonder.

But nah, it's too adictive. So, instead, you set your mind on trying to do a little filtering yourself and detect when  disasters are going on and people are tweeting about it. For that purpose you decide to start building a disaster detector.


After a lot of trouble extracting and classifying tweets, you come up with a dataset. 


Load the dataset and check its structure:


In [ ]:
df = pd.read_csv('datasets/twitter_disaster.csv')

df.head()

### Q1.a)

You realize, after all that you've learned in the NLP specialization, that it might be worth it to process your text a bit. 

Implement below a preprocessing function that does the following:

- tokenizes the text 
- lowercases the text 
- removes punctuation 
 

In [ ]:
def preprocess_data(df):
    """Returns a dataframe with preprocessed text as a new column
    
    Parameters:
        df: input dataframe, containing a "text" column
    
    Returns:
        preprocessed_df: output dataframe, with a new column "preprocessed_text" 
    """
    
    # YOUR CODE HERE
    raise NotImplementedError()
    
    return preprocessed_df


In [ ]:
preprocessed_df = preprocess_data(df)

assert hashlib.sha256(preprocessed_df['text'][200].encode()).hexdigest() == 'cb01387fe41613faa876fb6793179578747693d8dbfe57fd7a808376531f836f'
assert hashlib.sha256(preprocessed_df['text'][400].encode()).hexdigest() == '89869900237bf608f53e686bacb698075e76102791c7c7973dd6015730ca271c'
assert hashlib.sha256(preprocessed_df['text'][745].encode()).hexdigest() == '8ffa8f4f715c83feb96d9d8b427d90e6732e0e048e8d39b5d2694a797ed76c4e'

See how the preprocessed data looks like:

In [ ]:
preprocessed_df.head()

With your preprocessing looking nice and after spending hours just coming up with the dataset, you decide to call it a day and save your data. 

You save it in a different file (never know when you might need the raw one again) and go to sleep. 

**Note**: for further exercises you'll be using the pre-saved version of the data we provide.

### Q1.b)

In the morning you go back to your data. You start by loading your preprocessed data and split into a train and test set. 




In [ ]:
df = pd.read_csv('datasets/twitter_disaster_preprocessed.csv')

df.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['target'], test_size=0.3, random_state=42)

X_train.head()

You decide to move forward with some feature extraction, and go for a TFIDF tokenizer. However, you know that your space might be high-dimensional and you want to prepare for that. 

Implement a function that receives the number of desired features and returns the vectorized data together with the vectorizer and the space dimension:

In [ ]:
def extract_features(X_train, X_test, feature_limit=None):
    """Converts the provided text training and test data into 
    TF-IDF feature counts. Additionally, returns the used
    vectorizer and the space dimension
    
    Parameters:
        X_train: training data
        X_test: test data
        feature_limit: maximum number of features to use

    Returns:
        vectorizer: used tf-idf vectorizer 
        num_features: vectorizer actual number of features, for sanity check
        X_train_vec: processed training features
        X_test_vec: processed test features
    """
    
    # YOUR CODE HERE
    raise NotImplementedError()


In [ ]:
vectorizer, num_features, X_train_vec, X_test_vec = extract_features(X_train, X_test)
assert math.isclose(np.sum(X_train_vec.todense()[12, :]), 3.08383176160954)
assert num_features == 16801

vectorizer, num_features, X_train_vec, X_test_vec = extract_features(X_train, X_test, feature_limit=100)
assert math.isclose(X_train_vec.todense()[121, 12], 0.6280235685065689)
assert num_features == 100

vectorizer, num_features, X_train_vec, X_test_vec = extract_features(X_train, X_test, feature_limit=232)
assert math.isclose(np.sum(X_train_vec.todense()[300, :]), 2.906725853395745)
assert num_features == 232


### Q1.c)

Let's see what happens when we limit the number of features. What are the features the vectorizer selects?

Let's start by applying your function to obtain the vectorizer with 500 features: 

In [ ]:
vectorizer, num_features, X_train_vec, X_test_vec = extract_features(X_train, X_test, feature_limit=500)

Now create a function to obtain the selected features of the fitted vectorizer, sorted by their inverse document frequency (in descending order). In case of tie, this is, if several features have the same IDF score, the features should be sorted by alphabetical order.

In [ ]:
def get_ngrams_sorted_by_idf(fitted_vectorizer, top_features=30):
    """Returns a subset of features of a fitted TfidfVectorizer ordered by 
    their idf score
    
    Parameters:
        fitted_vectorizer (TfidfVectorizer): A fitted TfidfVectorizer
        top_features: Top features to return 
    
    Returns:
        ngrams_sorted (list): The top features of fitted_vectorizer sorted in descending order
                              by their idf score. In case of tie, the features should be sorted 
                              by alphabetical order
    """
# YOUR CODE HERE
raise NotImplementedError()


In [ ]:
assert hashlib.sha256(get_ngrams_sorted_by_idf(vectorizer, top_features=100)[12].encode()).hexdigest() == "aacd834b5cdc64a329e27649143406dd068306542988dfc250d6184745894849"
assert hashlib.sha256(get_ngrams_sorted_by_idf(vectorizer, top_features=100)[41].encode()).hexdigest() == "9e96274abcf3df0dfcd14617d6a14931c97fca364083918e38f8cfd20d136f0d"


Check the features with top inverse document frequency in your vectorizer:

In [ ]:
#show most specifc 30 features of the 100 selected features
get_ngrams_sorted_by_idf(vectorizer, top_features=30)

### Q1.d)

Not bad, but you remember learning about Chi squared,  method that will select features by relevance and you decide to compare how effective it is. 


In [ ]:
def extract_features_chi2(X_train, X_test, y_train, feature_limit=100):
    """Converts the provided text training and test data into 
    TF-IDF feature counts. Additionally, selects the best features
    with chi squared method. Finally, it returns the used
    vectorizer and the feature selector
    
    Parameters:
        X_train: training data
        X_test: test data
        y_train: training labels
        feature_limit: maximum number of features to use

    Returns:
        vectorizer: used tf-idf vectorizer 
        ch2: used feature selector
        X_train_ch2: processed training features
        X_test_ch2: processed test features
    """
    
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
vectorizer_ch2, ch2, X_train_ch2, X_test_ch2 = extract_features_chi2(X_train, X_test, y_train)
assert math.isclose(np.sum(X_train_ch2.todense()[12, :]), 0.15232491597861014)
assert ch2.k == 100

vectorizer_ch2, ch2, X_train_ch2, X_test_ch2 = extract_features_chi2(X_train, X_test, y_train, feature_limit=123)
assert math.isclose(np.sum(X_train_ch2.todense()[:, 122]), 36.32984514344287)
assert ch2.k == 123

vectorizer_ch2, ch2, X_train_ch2, X_test_ch2 = extract_features_chi2(X_train, X_test, y_train, feature_limit=232)
assert math.isclose(np.sum(X_train_ch2.todense()[300, :]), 1.1419313656030479)
assert ch2.k == 232

Once again, apply the extractor to our data and check the top relevant features

In [ ]:
vectorizer_ch2, ch2, X_train_ch2, X_test_ch2 = extract_features_chi2(X_train, X_test, y_train, feature_limit=30)

Now create a function similar to the one before to get the selected features ordered

In [ ]:
def get_chi2_ngrams_sorted(fitted_vectorizer, fitted_ch2_selector):
    """Returns the selected features (ngrams) given a vectorizer and 
    chi-squared selector
    
    Parameters:
        fitted_vectorizer (TfidfVectorizer): A fitted TfidfVectorizer
        fitted_ch2_selector (SelectKBest): A fitted chi squared selector 
    
    Returns:
        ngrams_sorted (list): The top features of fitted_vectorizer sorted in ascending order
                              by their idf score
    """
    # YOUR CODE HERE
    raise NotImplementedError()


Finally, check the features selected by chi squared:

In [ ]:
most_important_features = get_chi2_ngrams_sorted(vectorizer_ch2, ch2)

most_important_features

We can see how often each of the features selected appears in the tweets:

In [ ]:
for feature in most_important_features:
    print('Documents that contains the word(s) "%s"' % feature)
    print('----')
    docs = X_train.str.lower().str.contains(feature)
    print(str(y_train[docs].value_counts()) + '\n\n')

It should be visible that most features selected are linked to the "disaster" class (1).

## Q2.

### Time for some modelling

You finally decide to put everything together and get your first baseline. 

First of all, you figure you should use precision and recall to measure your models performance. Even though our data is not inbalanced it can give you more granularity over your target class - predicting a tweet is about disaster - even if a few tweets of the other class get mislabeled.

Implement a function to compute precision and recall given your predictions and true labels:


In [ ]:
def get_precision_recall(y_test, y_pred):
    """Returns the precision and recall of the disaster class (label = 1)
    
    Parameters:
        y_test (Series): Labels corresponding to X_test
        y_pred (Series): Predictions corresponding to X_test

    Returns:
        precision (float): The precision score of the disaster class (1) on the test data
        recall (float): The recall score of the disaster class (1) on the test data
    """
    
    # YOUR CODE HERE
    raise NotImplementedError()
    


In [ ]:
np.random.seed(42)
precision, recall = get_precision_recall(pd.Series(np.random.choice([0, 1], size=30)), pd.Series(np.random.choice([0, 1], size=30)))
assert precision == 0.4
assert recall == 0.375

vectorizer, np.random.seed(30)
precision, recall = get_precision_recall(pd.Series(np.random.choice([0, 1], size=30)), pd.Series(np.random.choice([0, 1], size=30)))
assert precision == 0.45
assert recall == 0.5294117647058824

vectorizer, np.random.seed(12)
precision, recall = get_precision_recall(pd.Series(np.random.choice([0, 1], size=30)), pd.Series(np.random.choice([0, 1], size=30)))
assert precision == 0.6
assert recall == 0.6


Now that your metrics are ready, you jump to the model. 

As stopwords appeared in the list of features also, you decide to provide an option to remove stop words.

Write up a function that receives the train and test data, vectorizes it, selects a given number of features with chi-squared, and finally trains a multinomial Naive Bayes model to classify the tweets. Expose your predictions precision and recall.

In [ ]:
def train_model_chi2_naive_bayes(X_train, y_train, X_test, y_test, max_features=100, stopwords=None):
    """Returns a fitted TfidfVectorizer, the chi squared selector, a naive bayes classifier
    and the test predictions computed with these
    
    Parameters:
        X_train (Series): Text data for training
        y_train (Series): Labels corresponding to X_train
        X_test (Series): Text data for testing
        y_test (Series): Labels corresponding to X_test
        max_features (int): maximum number of features to use
        stopwords (str): stopwords to use (if None does not use stopwords). eg 'english'

    Returns:
        vectorizer (TfidfVectorizer): TfidfVectorizer, fitted to X_train
        ch2 (SelectKBest): SelectKBest with score function chi2 and provided number of features, 
                           fitted to vectorized X_train
        clf (MultinomialNB): MultinomialNB classifier fitted to the feature-selected training data
        y_pred (Series): The predictions computed with our classifier
    """
    
    # YOUR CODE HERE
    raise NotImplementedError()
    
    return vectorizer, ch2, clf, y_pred

In [ ]:
vectorizer, ch2, clf, y_pred_200 = \
    train_model_chi2_naive_bayes(X_train, y_train, X_test, y_test, max_features=200, stopwords="english")
assert np.mean(y_pred_200) == 0.17075306479859895
assert np.sum(y_pred_200) == 390
assert ch2.k == 200
assert vectorizer.get_feature_names()[7424] == 'independence'


vectorizer, ch2, clf, y_pred_500 = \
    train_model_chi2_naive_bayes(X_train, y_train, X_test, y_test, max_features=500, stopwords="english")
assert np.mean(y_pred_500) == 0.2591943957968476
assert np.sum(y_pred_500) == 592
assert ch2.k == 500
assert vectorizer.get_feature_names()[7424] == 'independence'


vectorizer, ch2, clf, y_pred_1000 = \
    train_model_chi2_naive_bayes(X_train, y_train, X_test, y_test, max_features=1000, stopwords="english")
assert np.mean(y_pred_1000) == 0.3021015761821366
assert np.sum(y_pred_1000) == 690
assert ch2.k == 1000
assert vectorizer.get_feature_names()[7424] == 'independence'


Now apply your previous precision and recall function to check the metrics of each prediction:

In [ ]:
precision, recall = get_precision_recall(y_test, y_pred_200)     
print("\nPredictions with 200 features: ")
print("Precision: {}".format(precision))
print("Recall: {}".format(recall))

precision, recall = get_precision_recall(y_test, y_pred_500)      
print("\nPredictions with 500 features: ")
print("Precision: {}".format(precision))
print("Recall: {}".format(recall))

precision, recall = get_precision_recall(y_test, y_pred_1000)     
print("\nPredictions with 1000 features: ")
print("Precision: {}".format(precision))
print("Recall: {}".format(recall))


It seems we have a high precision but a low recall for predicting a disaster. But as you see, as we increase the number of features used we are sacrificing the precision and gaining on recall, probably widening the range of tweets we are classifying as a disaster. 

This makes sense given that most of the top selected features are linked to the disaster class.


## Q3.

You are pretty happy with your baseline but you're still using a pretty number of features to get a good precision/recall ratio. You now decide to move on to other techniques, such as SVD and PCA.


Start by writting up a similar function to the previous one to train your model, but use SVD and classify your model with Support Vector Classification.

In [ ]:
def train_model_svd_svm(X_train, y_train, X_test, y_test, num_features=100, seed=42):
    """Returns a fitted TfidfVectorizer, the truncated svd used, a support vector classifier
    and the test predictions computed with these
    
    Parameters:
        X_train (Series): Text data for training
        y_train (Series): Labels corresponding to X_train
        X_test (Series): Text data for testing
        y_test (Series): Labels corresponding to X_test
        num_features (int): maximum number of features to use
        seed (int): Seed to use for random state

    Returns:
        vectorizer (TfidfVectorizer): TfidfVectorizer, fitted to X_train
        svd (TruncatedSVD): TruncatedSVD with provided number of features as components
        clf (SVC): SVC classifier fitted to the feature-selected training data
        y_pred (Series): The predictions computed with our classifier
    """
    # YOUR CODE HERE
    raise NotImplementedError()
    
    return vectorizer, svd, clf, y_pred

In [ ]:
vectorizer, svd, clf, y_pred_30 = train_model_svd_svm(X_train, y_train, X_test, y_test, num_features=30, seed=42)
print("----- Computed for 30 features -----\n")
assert np.mean(y_pred_30) == 0.30122591943957966
assert np.sum(y_pred_30) == 688
assert svd.n_components == 30
assert vectorizer.get_feature_names()[5424] == 'exploit'


vectorizer, svd, clf, y_pred_50 = train_model_svd_svm(X_train, y_train, X_test, y_test, num_features=50, seed=42)
print("----- Computed for 50 features -----\n")
assert np.mean(y_pred_50) == 0.3261821366024518
assert np.sum(y_pred_50) == 745
assert svd.n_components == 50
assert vectorizer.get_feature_names()[5424] == 'exploit'

vectorizer, svd, clf, y_pred_100 = train_model_svd_svm(X_train, y_train, X_test, y_test, num_features=100, seed=42)
print("----- Computed for 100 features -----\n")
assert np.mean(y_pred_100) == 0.3445709281961471
assert np.sum(y_pred_100) == 787
assert svd.n_components == 100
assert vectorizer.get_feature_names()[5424] == 'exploit'

We can obtain the precision and recall as before and compare with the previous results:

In [ ]:
precision, recall = get_precision_recall(y_test, y_pred_30)     
print("\nPredictions with 30 features: ")
print("Precision: {}".format(precision))
print("Recall: {}".format(recall))

precision, recall = get_precision_recall(y_test, y_pred_50)      
print("\nPredictions with 50 features: ")
print("Precision: {}".format(precision))
print("Recall: {}".format(recall))

precision, recall = get_precision_recall(y_test, y_pred_100)     
print("\nPredictions with 100 features: ")
print("Precision: {}".format(precision))
print("Recall: {}".format(recall))

## Q4.

With much less features we get a better recall and only slightly lower precision, nice! Remember now some of the pros and cons of SVD and other dimensionality reduction techniques. While SVD performed better, likely because in theory it can capture information from more than 50 ngrams (whereas in chi-squared the number of features was in a direct correspondence with the number of ngrams), with SVD we lose interpretability and can't examine the most important features!

Now let's try PCA.

Write a similar function to Q3 but now using PCA to see how it compares to SVD

In [ ]:
def train_model_pca_svm(X_train, y_train, X_test, y_test, num_features=100, seed=42):
    """Returns a fitted TfidfVectorizer, the truncated svd used, a support vector classifier
    and the test predictions computed with these
    
    Parameters:
        X_train (Series): Text data for training
        y_train (Series): Labels corresponding to X_train
        X_test (Series): Text data for testing
        y_test (Series): Labels corresponding to X_test
        num_features (int): maximum number of features to use
        seed (int): Seed to use for random state

    Returns:
        vectorizer (TfidfVectorizer): TfidfVectorizer, fitted to X_train
        pca (PCA): PCA with provided number of features as components
        clf (SVC): SVC classifier fitted to the feature-selected training data
        y_pred (Series): The predictions computed with our classifier
    """
    # YOUR CODE HERE
    raise NotImplementedError()
    
    return vectorizer, pca, clf, y_pred

In [ ]:
vectorizer, pca, clf, y_pred_30 = train_model_pca_svm(X_train, y_train, X_test, y_test, num_features=30, seed=42)
print("----- Computed for 30 features -----\n")
assert np.mean(y_pred_30) == 0.30035026269702275
assert np.sum(y_pred_30) == 686
assert pca.n_components == 30
assert vectorizer.get_feature_names()[5424] == 'exploit'

vectorizer, pca, clf, y_pred_50 = train_model_pca_svm(X_train, y_train, X_test, y_test, num_features=50, seed=42)
print("----- Computed for 50 features -----\n")
assert np.mean(y_pred_50) == 0.3126094570928196
assert np.sum(y_pred_50) == 714
assert pca.n_components == 50
assert vectorizer.get_feature_names()[5424] == 'exploit'

vectorizer, pca, clf, y_pred_100 = train_model_pca_svm(X_train, y_train, X_test, y_test, num_features=100, seed=42)
print("----- Computed for 100 features -----\n")
assert np.mean(y_pred_100) == 0.33887915936952717
assert np.sum(y_pred_100) == 774
assert pca.n_components == 100
assert vectorizer.get_feature_names()[5424] == 'exploit'

We can obtain the precision and recall as before and compare with the previous results:

In [ ]:
precision, recall = get_precision_recall(y_test, y_pred_30)     
print("\nPredictions with 30 features: ")
print("Precision: {}".format(precision))
print("Recall: {}".format(recall))

precision, recall = get_precision_recall(y_test, y_pred_50)      
print("\nPredictions with 50 features: ")
print("Precision: {}".format(precision))
print("Recall: {}".format(recall))

precision, recall = get_precision_recall(y_test, y_pred_100)     
print("\nPredictions with 100 features: ")
print("Precision: {}".format(precision))
print("Recall: {}".format(recall))

## Q5.

Now we'll change gears a bit and look into word vectors. In Learning Notebook 3 we told you that word vectors can be visualized after being projected into 2D space, and we showed you this diagram:

<img src="./media/word-vectors-projection.png" width="600">

Now we'll try to combine what you've learned about word embeddings and PCA to make our own visualization!

In [ ]:
# load word embeddings

nlp = spacy.load('en_core_web_md')

### Q5.a)

First, to get comfortable with spacy, get the vector for the word "disaster" 

In [ ]:
# disaster_vector = 

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
assert math.isclose(disaster_vector.sum(), -6.004221, abs_tol=0.00001)
assert disaster_vector.shape[0] == 300

### Q5.b) 

Next, write a function that uses sklearn's PCA to reduce our vectors to a convenient number of dimensions for plotting. The function should return the reduced dimension word vectors.

In [ ]:
def reduce_word_vecs(vectors, random_state):
    """
    Returns PCA-reduced word vectors of the input vectors for plotting
    
    Parameters:
        vectors (np.array): Word vectors to be reduced
        random_state (int): random state to use in PCA

    Returns:
        reduced_vecs (np.array): Word vectors reduced to the number of dimensions
                                 suitable for plotting with given random_state
    """
    # YOUR CODE HERE
    raise NotImplementedError()
    
    return reduced_vecs

In [ ]:
test_vectors = np.array([[0.1, 0.2, 0.3, 0.4], [0.3, 0.5, 0.1, 0.7], [0.8, 0.6, 0.2, 0.4]])
reduced_vecs = reduce_word_vecs(test_vectors, random_state=42)

assert reduced_vecs.shape == (3,2)
assert math.isclose(reduced_vecs[1][1], 0.24736592153367926)
assert math.isclose(reduced_vecs[2][0], 0.43388622222454437)

Now, we'll create an array of ~75,000 of spacy's word vectors and use your PCA function to reduce them. If you're curious about using the full amount of word vectors, you can change the code to iterate over all vocab words - `list(nlp.vocab.strings)` - instead of our own `vocab_strings`, but beware it will use a lot of memory!

We'll also set a list of words that we'll plot later and force our set of vectors to include these.

In [ ]:
words_to_plot = ['banana', 'pineapple', 'mango', 'red', 'blue', 'yellow', 'woman', 'man', 'child', 'playing',
                 'reading', 'studying', 'nintendo', 'sony', 'xbox', 'sad', 'angry', 'bored']

In [ ]:
with open('datasets/word_subset.txt') as fwords:
    vocab_strings = fwords.read().splitlines()

full_vocab_vecs = []
for tok in vocab_strings:
    full_vocab_vecs.append(nlp.vocab.get_vector(tok))

vocab_array = np.array(full_vocab_vecs)

In [ ]:
print('Word vectors shape pre-PCA: {}'.format(vocab_array.shape))

full_vocab_reduced = reduce_word_vecs(vocab_array, random_state=42)

print('Word vectors shape after PCA: {}'.format(full_vocab_reduced.shape))

In [ ]:
assert math.isclose(full_vocab_reduced[100][0], -3.0880196, abs_tol=0.00001)
assert math.isclose(full_vocab_reduced[9999][0], -0.9480563, abs_tol=0.00001)

### Q5.c)

Time to plot! For this, we'll limit the visualized words to a small subset.

In [ ]:
coords = []
for word in words_to_plot:
    idx = vocab_strings.index(word)
    coords.append(full_vocab_reduced[idx])

coords_array = np.array(coords)

In [ ]:
plt.figure(num=None, figsize=(8, 8), dpi=80, facecolor='w', edgecolor='k')

plt.xlim(min([x for x in coords_array[:,0]]), max([x for x in coords_array[:,0]]))
plt.ylim(min([y for y in coords_array[:,1]]), max([y for y in coords_array[:,1]]))
plt.scatter(coords_array[:,0], coords_array[:,1])

for item, x, y in zip(words_to_plot, coords_array[:,0], coords_array[:,1]):
    plt.annotate(item, xy=(x, y), xytext=(-2, 2), textcoords='offset points', 
                 ha='right', va='bottom', color='purple', fontsize=14 )
plt.show()

The PCA seems to have worked! In the diagram we can see similar types of words closer together. But of course, take these visualizations with a grain of salt because it is practically impossible to preserve all distances in a high dimensional space in just 2 dimensions.

### Q5.d) 

As a final exercise, we'll look at some word similarities.

Write a function that returns the next closest word in terms of cosine similarity to a given word. If there are multiple words with the same highest similarity, return all of them.

Hint: you can use the already-imported `cosine_similarity` function from sklearn to compute cosine similarity.

In [ ]:
def closest_word(input_word, words_in_vocab, word_vectors):
    '''Returns a list of the closest word or words to the input word, based on cosine similarities
       of the word vectors given
    
    Parameters:
        input_word (string): Search for the closest words to this word
        words_in_vocab (list): Vocabulary associated with the vectors in word_vectors
        word_vectors (np.array): Word vectors associated with the strings in words_in_vocab

    Returns:
        closest_words_list (list): List of strings containing the closest word or words to
                                   input_word, based on cosine similarities of the word_vectors
    '''
    # YOUR CODE HERE
    raise NotImplementedError()
    return closest_words_list

In [ ]:
assert(hashlib.sha256(closest_word('nintendo', words_to_plot, coords_array)[0].encode()).hexdigest() == "f711da60664c04c146d7a47b722c38a8d0bf46c3f52c2084c5c8d1cb78138e73")
assert(hashlib.sha256(closest_word('playing', words_to_plot, coords_array)[0].encode()).hexdigest() == "435c149cbc6a5e5cc373cd33347d4c336a22160e06b7df61092b66e56f4d55ec")
assert(hashlib.sha256(closest_word('pineapple', words_to_plot, coords_array)[0].encode()).hexdigest() == "6815f3c300383519de8e437497e2c3e97852fe8d717a5419d5aafb00cb43c494")

In [ ]:
print("\nClosest words to nintendo:")
print(closest_word('nintendo', words_to_plot, coords_array))

print("\nClosest words to playing:")
print(closest_word('playing', words_to_plot, coords_array))

print("\nClosest words to pineapple:")
print(closest_word('pineapple', words_to_plot, coords_array))